<a href="https://colab.research.google.com/github/E1250/ZTM-Courses/blob/main/01_pytorch_workflow_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. PyTorch Workflow Exercise Template

The following is a template for the PyTorch workflow exercises.

It's only starter code and it's your job to fill in the blanks.

Because of the flexibility of PyTorch, there may be more than one way to answer the question.

Don't worry about trying to be *right* just try writing code that suffices the question.

You can see one form of [solutions on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/extras/solutions) (but try the exercises below yourself first!).

In [42]:
# Import necessary libraries
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
sns.set()

In [43]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

## 1. Create a straight line dataset using the linear regression formula (`weight * X + bias`).
  * Set `weight=0.3` and `bias=0.9` there should be at least 100 datapoints total. 
  * Split the data into 80% training, 20% testing.
  * Plot the training and testing data so it becomes visual.

Your output of the below cell should look something like:
```
Number of X samples: 100
Number of y samples: 100
First 10 X & y samples:
X: tensor([0.0000, 0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800,
        0.0900])
y: tensor([0.9000, 0.9030, 0.9060, 0.9090, 0.9120, 0.9150, 0.9180, 0.9210, 0.9240,
        0.9270])
```

Of course the numbers in `X` and `y` may be different but ideally they're created using the linear regression formula.

In [52]:
# Create the data parameters
weight = 0.3
bias = 0.9

start = 0
end = 1
step = 1/100

X = torch.arange(start,end,step).unsqueeze(dim=1)
y = weight*X+bias


# Make X and y using linear regression feature
print(f"Number of X samples: {len(X)}")
print(f"Number of y samples: {len(y)}")
print(f"First 10 X & y samples:\nX: {X[:10]}\ny: {y[:10]}")

Number of X samples: 100
Number of y samples: 100
First 10 X & y samples:
X: tensor([[0.0000],
        [0.0100],
        [0.0200],
        [0.0300],
        [0.0400],
        [0.0500],
        [0.0600],
        [0.0700],
        [0.0800],
        [0.0900]])
y: tensor([[0.9000],
        [0.9030],
        [0.9060],
        [0.9090],
        [0.9120],
        [0.9150],
        [0.9180],
        [0.9210],
        [0.9240],
        [0.9270]])


In [53]:
# Split the data into training and testing
split = int(0.8*len(X))

X_train , y_train = X[:split],y[:split]
X_test , y_test = X[split:],y[split:]

len(X_train), len(X_test) ,len(y_train), len(y_test)

(80, 20, 80, 20)

In [54]:
# Plot the training and testing data
def plotter(pred):
  plt.plot(X_train,y_train)
  plt.plot(X_test,y_test)
  plt.plot(pred,y_test)
  plt.title("Plotting the Training And Testing Data")
  plt.xlabel("Testing")
  plt.ylabel("Predicting")
  plt.show()

## 2. Build a PyTorch model by subclassing `nn.Module`. 
  * Inside should be a randomly initialized `nn.Parameter()` with `requires_grad=True`, one for `weights` and one for `bias`. 
  * Implement the `forward()` method to compute the linear regression function you used to create the dataset in 1. 
  * Once you've constructed the model, make an instance of it and check its `state_dict()`.
  * **Note:** If you'd like to use `nn.Linear()` instead of `nn.Parameter()` you can.

In [55]:
# Create PyTorch linear regression model by subclassing nn.Module
class LinearRegressionModel(nn.Module):
  def __init__(self):
    super(LinearRegressionModel, self).__init__()
    self.linearModel = nn.Linear(in_features=1,out_features=1)
  def forward(self,x:torch.Tensor) -> torch.Tensor:
    return self.linearModel(x)

In [56]:
# Instantiate the model and put it to the target device
model = LinearRegressionModel()
model.to(device)
model.state_dict()

OrderedDict([('linearModel.weight', tensor([[-0.2808]])),
             ('linearModel.bias', tensor([0.8865]))])

## 3. Create a loss function and optimizer using `nn.L1Loss()` and `torch.optim.SGD(params, lr)` respectively. 
  * Set the learning rate of the optimizer to be 0.01 and the parameters to optimize should be the model parameters from the model you created in 2.
  * Write a training loop to perform the appropriate training steps for 300 epochs.
  * The training loop should test the model on the test dataset every 20 epochs.

In [57]:
# Create the loss function and optimizer
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model.parameters(),lr = 0.01)

In [62]:
# Training loop

# Train model for 300 epochs
epochs = 300


# Send data to target device
X_train.to(device)
X_test.to(device)
y_train.to(device)
y_test.to(device)

for epoch in range(epochs):
  ### Training

  # Put model in train mode
  model.train()  

  # 1. Forward pass
  pred = model(X_train)

  # 2. Calculate loss
  loss = loss_fn(pred,y_train)

  # 3. Zero gradients
  optimizer.zero_grad()

  # 4. Backpropagation
  loss.backward()

  # 5. Step the optimizer
  optimizer.step()

  ### Perform testing every 20 epochs
  if epoch % 20 == 0:

    # Put model in evaluation mode and setup inference context 
    model.eval()
    with torch.inference_mode():
      # 1. Forward pass
      pred = model(X_test)
      
      # 2. Calculate test loss
      test_loss = loss_fn(pred,y_test)

      # Print out what's happening
      print(f"Epoch: {epoch} | Train loss: {loss:.3f} | Test loss: {test_loss:.3f}")

Epoch: 0 | Train loss: 0.220 | Test loss: 0.493
Epoch: 20 | Train loss: 0.111 | Test loss: 0.311
Epoch: 40 | Train loss: 0.090 | Test loss: 0.232
Epoch: 60 | Train loss: 0.082 | Test loss: 0.196
Epoch: 80 | Train loss: 0.075 | Test loss: 0.176
Epoch: 100 | Train loss: 0.068 | Test loss: 0.160
Epoch: 120 | Train loss: 0.061 | Test loss: 0.144
Epoch: 140 | Train loss: 0.054 | Test loss: 0.127
Epoch: 160 | Train loss: 0.047 | Test loss: 0.111
Epoch: 180 | Train loss: 0.040 | Test loss: 0.095
Epoch: 200 | Train loss: 0.034 | Test loss: 0.079
Epoch: 220 | Train loss: 0.027 | Test loss: 0.062
Epoch: 240 | Train loss: 0.020 | Test loss: 0.046
Epoch: 260 | Train loss: 0.013 | Test loss: 0.030
Epoch: 280 | Train loss: 0.006 | Test loss: 0.014


## 4. Make predictions with the trained model on the test data.
  * Visualize these predictions against the original training and testing data (**note:** you may need to make sure the predictions are *not* on the GPU if you want to use non-CUDA-enabled libraries such as matplotlib to plot).

In [ ]:
# Make predictions with the model


In [ ]:
# Plot the predictions (these may need to be on a specific device)


## 5. Save your trained model's `state_dict()` to file.
  * Create a new instance of your model class you made in 2. and load in the `state_dict()` you just saved to it.
  * Perform predictions on your test data with the loaded model and confirm they match the original model predictions from 4.

In [ ]:
from pathlib import Path

# 1. Create models directory 


# 2. Create model save path 

# 3. Save the model state dict


In [ ]:
# Create new instance of model and load saved state dict (make sure to put it on the target device)


In [ ]:
# Make predictions with loaded model and compare them to the previous
